Algorithm 1(eliminate points-m): 
    Input: p1,p2,... , pn' (in order) where n' is the number of points in the stream. 
    Output: Skyline points S' 
    1: Let x = 24m. 
    2: Pass 1: For j : 1, 2, ..., x, let p'j be a point picked uniformly at random from the stream. 
    Let S be the set of such points.
    3: Pass 2: 
    4: for i = 1, ..., n' do 
    5: for any p'j, if pi dominates p'j then p'j:=pi
    6: end for 
    7: Let S'={p'1,p'2,...,p'x}.
    8: Pass 3: 
            Delete from stream all points in S' and all points dominated by any point in S'.
    9: return S' 

In [4]:
from random import randint
from math import log 

class stream_creation:
    def __init__(self, stream_size, dimension):
        self._stream_size = n 
        self._dimension = dimension 
        
    def stream_creation(_dimension,_stream_size):
        stream = ()
        for point in range(_stream_size):
            coordinates = ()
            for coordinate in range(_dimension):
                coordinates = coordinates + (randint(1,100),)
            stream = stream +(coordinates,)
        return(stream)

class eliminate_points_algorithm:
    def __init__(self,amount_skylinepoints, stream): 
        self._amount_skylinepoints = amount_skylinepoints
        self._stream = stream 

    def transform_stream_tuple_in_list(_stream):
        stream_list= [] 
        for component in _stream:
            stream_list.append(component)
        return(stream_list)

    def reservoir_sampling(stream,_amount_skylinepoints):
        S = []
        if len(stream)>24*_amount_skylinepoints:
            size_sample = 24*_amount_skylinepoints
            for initial_sample in range(size_sample):
                S.append(stream[initial_sample])
            
            for address_candidate_sample, candidate_sample in enumerate(stream): 
                chance = randint(0,address_candidate_sample)
                if chance < size_sample:
                    S[chance] = candidate_sample
        else:
            for initial_sample in range(len(stream)):
                S.append(stream[initial_sample])
        return(S) 

    def random_domination_from_stream_on_reservoir(stream, S):  
        for candidate in stream:
            chance = randint(0,len(S)-1)  
            if (candidate[0]>= S[chance][0]) and (candidate[1]>=S[chance][1]) and(candidate[0]> S[chance][0] or candidate[1]>S[chance][1]):
                S[chance] = candidate
        return(S)

    def elimination_S_components_from_stream(stream,S):
        for proponent in S:
            for opponent in stream:
                if opponent == proponent:
                    stream.remove(opponent)
        return(stream)
    
    def elimination_dominated_by_S_components_from_stream(stream,S):            
        for victim in stream:
            for candidate in S:
                points = 0
                if victim in stream:
                    if candidate[0]>= victim[0] and candidate[1]>=victim[1] and(candidate[0]> victim[0] or candidate[1]>victim[1]):
                        stream.remove(victim)
        return(stream)



Algorithm 2 (Streaming RAND): 
    1: Let n be the number of points in the input stream. 
    Let m' = 1. 
    2: while the input stream is not empty do: 
    3: let n' be the current number of points in the stream 
    4: Call eliminate points (m'log(nlogn))
    5: If more than n'/2 points are left in the stream, m' = 2 m'
    6: end while 
    Remark: In case the stream cannot be changed, we do not have to actually delete points from stream. 
    We only keep the skyline points found so far and consider only points in the stream that is not dominated by any found skyline points. 

In [7]:
# this is a system where we begin with the stream from what hasn't been put in the reservoir nor dominated in the last round 
# here we double the total pool or less 
class streaming_RAND:
    def __init__(self, stream):
        self._stream = stream 

    def find_skylinepoints(stream):
        remaining_stream = eliminate_points_algorithm.transform_stream_tuple_in_list(stream)
        n = len(stream)
        skylinepoints = 1
        total_skylinepoints = 1
        previous_skylinepoints = 0
        total_pool = []
        dominated_stream = [1]
        length_previous_stream = len([])
        while len(remaining_stream) > 0:  
            non_dominated_reservoir = eliminate_points_algorithm.reservoir_sampling(remaining_stream,int(skylinepoints*log(n*log(n))//1))
            dominated_reservoir = eliminate_points_algorithm.random_domination_from_stream_on_reservoir(remaining_stream,non_dominated_reservoir)
            stream_list_with_no_reservoir_points = eliminate_points_algorithm.elimination_S_components_from_stream(remaining_stream,dominated_reservoir)
            dominated_stream = eliminate_points_algorithm.elimination_dominated_by_S_components_from_stream(stream_list_with_no_reservoir_points,dominated_reservoir)
            if length_previous_stream >= 2*len(dominated_stream):
                total_skylinepoints = total_skylinepoints*2
            else:
                total_skylinepoints += 1
            previous_skylinepoints += skylinepoints 
            skylinepoints = total_skylinepoints-previous_skylinepoints
            remaining_stream = dominated_stream
            length_previous_stream = len(dominated_stream)
            for component in dominated_reservoir:
                total_pool.append(component)
            print(skylinepoints//2, len(remaining_stream),len(dominated_reservoir),len(total_pool))

In [3]:
stream = stream_creation.stream_creation(2,10000)
print(streaming_RAND.find_skylinepoints(stream)) 

0 3443 264 264
1 1297 264 528
2 364 528 1056
4 28 364 1420
8 1 28 1448
16 0 1 1449
None


In [10]:
# this is a system where we begin with the stream from what hasn't been put in the reservoir nor dominated in the last round 
# here we double the total pool or less 
class streaming_RAND:
    def __init__(self, stream):
        self._stream = stream 

    def find_skylinepoints_fixed_window(stream,window=len(stream)-1):
        remaining_stream = eliminate_points_algorithm.transform_stream_tuple_in_list(stream)
        total_pool = []
        dominated_stream = [1]
        while len(remaining_stream) > 0:  
            non_dominated_reservoir = eliminate_points_algorithm.reservoir_sampling(remaining_stream,int(window//24))
            dominated_reservoir = eliminate_points_algorithm.random_domination_from_stream_on_reservoir(remaining_stream,non_dominated_reservoir)
            stream_list_with_no_reservoir_points = eliminate_points_algorithm.elimination_S_components_from_stream(remaining_stream,dominated_reservoir)
            dominated_stream = eliminate_points_algorithm.elimination_dominated_by_S_components_from_stream(stream_list_with_no_reservoir_points,dominated_reservoir)
            remaining_stream = dominated_stream
            length_previous_stream = len(dominated_stream)
            for adress_component,component in enumerate(dominated_reservoir):
                for adress_other_component, other_component in enumerate(dominated_reservoir):
                    if adress_component!=adress_other_component:
                        if component == other_component:
                            dominated_reservoir.remove(other_component)
            for component in dominated_reservoir:
                total_pool.append(component)
            print( len(remaining_stream),len(dominated_reservoir),len(total_pool))

In [ ]:
stream = stream_creation.stream_creation(2,10000)
print(streaming_RAND.find_skylinepoints_fixed_window(stream)) 